In [203]:
import pandas as pd
from country_named_entity_recognition import find_countries
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import itertools
from tqdm.auto import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lab000\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lab000\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [204]:
label_name_list = ["PATIENT","DOCTOR","USERNAME","PROFESSION","ROOM","DEPARTMENT","HOSPITAL",
              "ORGANIZATION","STREET","CITY","STATE","COUNTRY","ZIP","LOCATION-OTHER",
              "AGE","DATE","TIME","DURATION","SET","PHONE","FAX","EMAIL","URL","IPADDR",
              "SSN","MEDICALRECORD","HEALTHPLAN","ACCOUNT","LICENSE","VECHICLE","DEVICE",
              "BIOID","IDNUM","PHI"]

id_to_label = dict(enumerate(label_name_list))
label_to_id = {v: k for k, v in id_to_label.items()}

In [205]:
def find_all(a_string, sub):
    result = []
    k = 0
    while k < len(a_string):
        k = a_string.find(sub, k)
        if k == -1:
            return result
        else:
            start = k
            end = k + len(sub)

            result.append(f"{start} {end}")
            # k += 1 #change to k += len(sub) to not search overlapping results
            k += len(sub)

    return result

In [206]:
path = "./NER_Dataset/private_data"
final_test_df = pd.read_csv(f'{path}/opendid_test.tsv',sep='\t',names = ["fid","idx","text"])

In [207]:
final_test_df

,fid,idx,text
0,1097,1,433475.RDC
1,1097,12,"Timmins, ELDEN"
2,1097,27,"43J47561,43J47561"
3,1097,46,Last edited : 7/9/2063 Page: 2
4,1097,78,CLINICAL:
...,...,...,...
78975,HSA0590,6664,Distal small bowel:
78976,HSA0590,6684,- 22mm benign polyp with features favouring an...
78977,HSA0590,6773,"- 7mm neuroendocrine tumour, grade 1 (carcinoi..."
78978,HSA0590,6972,- 1 lymph node with no evidence of malignancy


# LOCAION-OTHER

In [208]:
LOCAION_OTHER_data_list = []
LOCAION_OTHER_key_word_list = ["&quot;","PO "]
for ind in final_test_df.index:
    text = final_test_df.loc[ind].text
    text = str(text)

    for key_word in LOCAION_OTHER_key_word_list:
        if key_word in text:
            LOCAION_OTHER_data_list.append(ind)

In [209]:
LOCAION_OTHER_test_df = final_test_df.loc[LOCAION_OTHER_data_list].reset_index(drop=True)
print(LOCAION_OTHER_test_df)
print("LOCAION-OTHER筆數約:", len(LOCAION_OTHER_test_df))

          fid   idx                                               text
0        2465    68                                        PO BOX 1061
1        5243   415  Please send copies of pathology to Dr SIA Perc...
2   file37972   493  &quot;UTERUS, PERITONEAL DISEASE, SPLEEN, OMEN...
3   file37972  5346                       &quot;RIGHT DIAPHRAGM&quot;.
4   file41008   476      &quot;SENTINEL NODE BIOPSY LEFT BREAST&quot;.
5   file41008   632  FROZEN SECTION REPORT:  &quot;Negative&quot;. ...
6   file41008   722       &quot;WIDE LOCAL EXCISION LEFT BREAST&quot;.
7   file41008  2127      &quot;LEFT AXILLARY TISSUE LEFT BREAST&quot;.
8   file41429   375  Specimen labelled &quot;RE-EXCISION MARGINS IN...
9   file42064   385  &quot;RE-EXCISION OF ANTEROMEDIAL MARGIN, Shor...
10  file50077   308  &quot;OMENTUM, RIGHT COLON, SPLEEN, GALLBLADDE...
LOCAION-OTHER筆數約: 11


In [210]:
LOCAION_OTHER_text_list = []
LOCAION_OTHER_PHI_text_list = []
LOCAION_OTHER_PHI_list = []
LOCAION_OTHER_fid_list = []
LOCAION_OTHER_idx_list = []
for ind in LOCAION_OTHER_test_df.index:
    text = LOCAION_OTHER_test_df.loc[ind].text
    text = str(text)
    fid = LOCAION_OTHER_test_df.loc[ind].fid
    idx = LOCAION_OTHER_test_df.loc[ind].idx
    for key_word in LOCAION_OTHER_key_word_list:
        sub_text_index = find_all(text,key_word)
        if len(sub_text_index) != 0:
            if key_word == "PO ":
                if(len(text)<=12):
                    LOCAION_OTHER_text_list.append(text)
                    LOCAION_OTHER_PHI_text_list.append(text)
                    LOCAION_OTHER_fid_list.append(fid)
                    LOCAION_OTHER_idx_list.append(idx)
                    LOCAION_OTHER_PHI_list.append(label_to_id["LOCATION-OTHER"])
                    continue
                    
                else:
                    for sub_index in range(0, len(sub_text_index), 1):
                        cur_index = int(sub_text_index[sub_index].split(" ")[0])
                        LOCAION_OTHER_text_list.append(text)
                        LOCAION_OTHER_PHI_text_list.append(text[cur_index:cur_index+11])
                        LOCAION_OTHER_fid_list.append(fid)
                        LOCAION_OTHER_idx_list.append(idx)
                        LOCAION_OTHER_PHI_list.append(label_to_id["LOCATION-OTHER"])
                    continue
                    

            for sub_index in range(0, len(sub_text_index), 2):
                cur_index = int(sub_text_index[sub_index].split(" ")[0])
                next_index = int(sub_text_index[sub_index + 1].split(" ")[1])
                
                LOCAION_OTHER_text_list.append(text)
                LOCAION_OTHER_PHI_text_list.append(text[cur_index:next_index])
                LOCAION_OTHER_fid_list.append(fid)
                LOCAION_OTHER_idx_list.append(idx)
                LOCAION_OTHER_PHI_list.append(label_to_id["LOCATION-OTHER"])



In [211]:
LOCAION_OTHER_result = {'fid':LOCAION_OTHER_fid_list,'idx':LOCAION_OTHER_idx_list,'text':LOCAION_OTHER_text_list,'label':LOCAION_OTHER_PHI_list,'label_text':LOCAION_OTHER_PHI_text_list}

answer_LOCAION_OTHER_test_df = pd.DataFrame(data=LOCAION_OTHER_result)

In [212]:
answer_LOCAION_OTHER_test_df.to_csv('./Search_Phi_Result/LOCATION-OTHER/ans_LOCAION_OTHER_test_df.txt', sep='\t', index=False)

# DURATION

In [213]:
DURATION_data_list = []
DURATION_data_key_list = []
DURATION_key_word_list = ["week","month","year"," yr"]
for ind in final_test_df.index:
    text = final_test_df.loc[ind].text
    text = str(text)

    for key_word in DURATION_key_word_list:
        if key_word in text:
            DURATION_data_list.append(ind)
            if key_word==" yr":
                key_word=key_word.strip()
            DURATION_data_key_list.append(key_word)



In [214]:
DURATION_test_df = final_test_df.loc[DURATION_data_list].reset_index(drop=True)
DURATION_test_df["key_word_text"] = DURATION_data_key_list
print(DURATION_test_df)
print("DURATION筆數約:", len(DURATION_test_df))

          fid    idx                                               text  \
0        1192    245  L TKR last year, had MRSAinfection in the join...   
1        1198    377               Also ? met (gastric ca 8 years ago).   
2        1250    313  Low grade glioma on previous biopsy this year ...   
3        1482    328  69 yr old woman with previous diagnosis of bow...   
4        1633    324  2 months of progressive decreasing visual acuity.   
5        1633    375  18 months hypocortisolaemia   Suprasellar lesion.   
6        2080    526  69 year old female with adnexal mass for TAH, ...   
7        2088    350                                      50 yr pkt hx.   
8        2110   3520  The case was allocated to me in Professor Ratl...   
9        2115  18875  62 year old male undergoing resection of a rig...   
10       2408    378                                  68 years old man.   
11       2418    127                      Bg:  Wilms tumour 15 yrs ago.   
12       2418    159  Add

In [215]:
DURATION_text_list = []
DURATION_phi_text_list = []
DURATION_phi_list = []
DURATION_fid_list = []
DURATION_idx_list = []

DURATION_key_phrase_list = ["JJ","DT"]
for ind in DURATION_test_df.index:
    text = DURATION_test_df.loc[ind].text
    text_token = nltk.word_tokenize(text)
    key_word_text = DURATION_test_df.loc[ind].key_word_text
    fid = DURATION_test_df.loc[ind].fid
    idx = DURATION_test_df.loc[ind].idx
    result = nltk.pos_tag(text_token)
    for pos, sub_result in enumerate(result):
        pos_text = sub_result[0]
        pos_tag = sub_result[1]
        if key_word_text in pos_text:
            previos_text = result[pos-1][0]
            previos_pos_tag = result[pos-1][1]
            if previos_pos_tag not in DURATION_key_phrase_list:
                # pass
                # number = result[pos-1][0]
                # print(previos_text+" "+pos_text)
                label_text = previos_text+" "+pos_text
                DURATION_text_list.append(text)
                DURATION_phi_text_list.append(label_text)
                DURATION_phi_list.append(label_to_id["DURATION"])
                DURATION_fid_list.append(fid)
                DURATION_idx_list.append(idx)
            # print(sub_result)
    # print(result)

In [216]:
DURATION_result = {'fid':DURATION_fid_list,'idx':DURATION_idx_list,'text':DURATION_text_list,'label':DURATION_phi_list,'label_text':DURATION_phi_text_list}

answer_DURATION_test_df = pd.DataFrame(data=DURATION_result)

In [217]:
answer_DURATION_test_df.to_csv('./Search_Phi_Result/DURATION/answer_DURATION_test_df.txt', sep='\t', index=False)

# URL

In [218]:
URL_data_list = []
URL_key_word_list = ["http"]
for ind in final_test_df.index:
    text = final_test_df.loc[ind].text
    text = str(text)

    for key_word in URL_key_word_list:
        if key_word in text:
            URL_data_list.append(ind)

In [219]:
URL_test_df = final_test_df.loc[URL_data_list].reset_index(drop=True)
print(URL_test_df)
print("URL筆數約:", len(URL_test_df))

    fid    idx                                               text
0  1975   4038  For a full list of the 238 mutations tested, p...
1  2483  10418  For a full list of the mutations tested, pleas...
2  5029   5565  * Catalogue of Somatic Mutations in Cancer: ht...
3  5123   8871  This information is entered into an MD Anderso...
URL筆數約: 4


In [220]:
URL_text_list = []
URL_PHI_text_list = []
URL_PHI_list = []
URL_fid_list = []
URL_idx_list = []
for ind in URL_test_df.index:
    text = URL_test_df.loc[ind].text
    text = str(text)
    fid = URL_test_df.loc[ind].fid
    idx = URL_test_df.loc[ind].idx
    for key_word in URL_key_word_list:
        if key_word in text:
            URL_text_list.append(text)
            URL_PHI_text_list.append(key_word)
            URL_PHI_list.append(label_to_id["URL"])
            URL_fid_list.append(fid)
            URL_idx_list.append(idx)


In [221]:
URL_result = {'fid':URL_fid_list,'idx':URL_idx_list,'text':URL_text_list,'label':URL_PHI_list,'label_text':URL_PHI_text_list}

answer_URL_test_df = pd.DataFrame(data=URL_result)

In [222]:
answer_URL_test_df.to_csv('./Search_Phi_Result/URL/ans_URL_test_df.txt', sep='\t', index=False)

# Set

In [223]:
Set_data_list = []
Set_key_word_list = ["twice"]
for ind in final_test_df.index:
    text = final_test_df.loc[ind].text
    text = str(text)

    for key_word in Set_key_word_list:
        if text.find(key_word)!= -1:
            Set_data_list.append(ind)

In [224]:
Set_test_df = final_test_df.loc[Set_data_list].reset_index(drop=True)
print(Set_test_df)
print("Set筆數約:", len(Set_test_df))

         fid   idx                                               text
0       2070  2666  The sample was tested twice with the appropria...
1       2077  2401  The sample was tested twice with the appropria...
2       2387  3717  DNA was extracted from paraffin tissue from Ra...
3  file32052  4108  Number found:   11 (specimen examined twice fo...
Set筆數約: 4


In [225]:
Set_text_list = []
Set_PHI_text_list = []
Set_PHI_list = []
Set_fid_list = []
Set_idx_list = []
for ind in Set_test_df.index:
    text = Set_test_df.loc[ind].text
    text = str(text)
    count_r_n = find_all(text,'\r\n')
    if len(count_r_n)>0:
        fid_idx_text = str(text.split('\r\n')[-1])
        
        fid = str(fid_idx_text.split("\t")[0])
        idx = int(fid_idx_text.split("\t")[1])
        text = str(fid_idx_text.split("\t")[2])

    else:
        fid = Set_test_df.loc[ind].fid
        idx = Set_test_df.loc[ind].idx
    for key_word in Set_key_word_list:
        if text.find(key_word)!= -1:
            Set_text_list.append(text)
            Set_PHI_text_list.append(key_word)
            Set_PHI_list.append(label_to_id["SET"])
            Set_fid_list.append(fid)
            Set_idx_list.append(idx)


In [226]:
URL_result = {'fid':Set_fid_list,'idx':Set_idx_list,'text':Set_text_list,'label':Set_PHI_list,'label_text':Set_PHI_text_list}

answer_Set_test_df = pd.DataFrame(data=URL_result)

In [227]:
answer_Set_test_df

,fid,idx,text,label,label_text
0,2070,2666,The sample was tested twice with the appropria...,18,twice
1,2077,2401,The sample was tested twice with the appropria...,18,twice
2,2387,3929,The sample was tested twice with the appropria...,18,twice
3,file32052,4108,Number found: 11 (specimen examined twice fo...,18,twice


In [228]:
answer_Set_test_df.to_csv('./Search_Phi_Result/Set/ans_Set_test_df.txt', sep='\t', index=False)

# COUNTRY

In [229]:
final_test_df

,fid,idx,text
0,1097,1,433475.RDC
1,1097,12,"Timmins, ELDEN"
2,1097,27,"43J47561,43J47561"
3,1097,46,Last edited : 7/9/2063 Page: 2
4,1097,78,CLINICAL:
...,...,...,...
78975,HSA0590,6664,Distal small bowel:
78976,HSA0590,6684,- 22mm benign polyp with features favouring an...
78977,HSA0590,6773,"- 7mm neuroendocrine tumour, grade 1 (carcinoi..."
78978,HSA0590,6972,- 1 lymph node with no evidence of malignancy


In [230]:
country_ind_list = []
country_match = []
for ind in final_test_df.index:
  text = final_test_df.loc[ind].text
  text = str(text)
  result = find_countries(text)

  if len(result) == 0:
    continue
  else:
    country_ind_list.append(ind)
    start = result[0][-1].span()[0]
    end = result[0][-1].span()[1]
    country_match.append(text[start:end])

In [231]:
Country_test_df = final_test_df.loc[country_ind_list].reset_index(drop=True)

In [232]:
country_text_list = []
country_ans_list = []
country_PHI_list = []
country_fid_list = []
country_idx_list = []
key_phrase = ['NNP','NN', 'JJ', 'POS', 'VBD']
for pos, ind in enumerate(country_ind_list):
    text = final_test_df.loc[ind].text
    text_token = nltk.word_tokenize(text)
    result = nltk.pos_tag(text_token)
    fid = final_test_df.loc[ind].fid
    idx = final_test_df.loc[ind].idx
    # print(result)
    country = country_match[pos]
    # print(country)
    for pos, sub_result in enumerate(result):
        
        # print(sub_result)
        if sub_result[0] == country:
            if pos==0:
                if (result[pos+1][1] not in  key_phrase):
                    country_text_list.append(text)
                    country_ans_list.append(country)
                    country_PHI_list.append(label_to_id["COUNTRY"])
                    country_fid_list.append(fid)
                    country_idx_list.append(idx)

            elif pos==(len(result)-1):
                if (result[pos-1][1] not in  key_phrase):
                    country_text_list.append(text)
                    country_ans_list.append(country)
                    country_PHI_list.append(label_to_id["COUNTRY"])
                    country_fid_list.append(fid)
                    country_idx_list.append(idx)
            else:

                # if result[pos-1][1] != 'NNP':
                if (result[pos-1][1] not in  key_phrase) and (result[pos+1][1] not in  key_phrase):
                    if result[pos+1][0] == '(' :
                        if result[pos+2][1] == 'NN':
                            continue
                    country_text_list.append(text)
                    country_ans_list.append(country)
                    country_PHI_list.append(label_to_id["COUNTRY"])
                    country_fid_list.append(fid)
                    country_idx_list.append(idx)
                    print(text)
                    print(result)


    

In [233]:
Country_result = {'fid':country_fid_list,'idx':country_idx_list,'text':country_text_list,'label':country_PHI_list,'label_text':country_ans_list}

answer_Country_test_df = pd.DataFrame(data=Country_result)

In [234]:
answer_Country_test_df.to_csv('./Search_Phi_Result/Country/ans_Country_test_df.txt', sep='\t', index=False)

In [235]:
answer_full_test_df = pd.concat([answer_LOCAION_OTHER_test_df,answer_DURATION_test_df,answer_URL_test_df,answer_Set_test_df,answer_Country_test_df], ignore_index=True)

In [236]:
answer_full_test_df

,fid,idx,text,label,label_text
0,2465,68,PO BOX 1061,13,PO BOX 1061
1,5243,415,Please send copies of pathology to Dr SIA Perc...,13,PO Box 1013
2,file37972,493,"&quot;UTERUS, PERITONEAL DISEASE, SPLEEN, OMEN...",13,"&quot;UTERUS, PERITONEAL DISEASE, SPLEEN, OMEN..."
3,file37972,5346,&quot;RIGHT DIAPHRAGM&quot;.,13,&quot;RIGHT DIAPHRAGM&quot;
4,file41008,476,&quot;SENTINEL NODE BIOPSY LEFT BREAST&quot;.,13,&quot;SENTINEL NODE BIOPSY LEFT BREAST&quot;
...,...,...,...,...,...
56,file32052,4108,Number found: 11 (specimen examined twice fo...,18,twice
57,5088,14,"Helde, ISRAEL",11,ISRAEL
58,5343,13,"Laos, Raya",11,Laos
59,5372,37,"Nyswonger, Israel",11,Israel


# Infer

In [237]:
lengths = []
tk0 = tqdm(answer_full_test_df['text'].fillna("").values, total=len(answer_full_test_df))

final_pre_text = []

for index,text in enumerate(tk0):
    
    
    pre_label = answer_full_test_df.loc[index]["label"]
    fid = answer_full_test_df.loc[index]["fid"]
    idx = answer_full_test_df.loc[index]["idx"]
    pre_word_text = answer_full_test_df.loc[index]["label_text"]


    offset_idx = text.find(pre_word_text)
    print(f"{fid}\t{id_to_label[pre_label]}\t{idx+offset_idx}\t{idx+offset_idx+len(pre_word_text)}\t{pre_word_text}")
    final_pre_text.append(f"{fid}\t{id_to_label[pre_label]}\t{idx+offset_idx}\t{idx+offset_idx+len(pre_word_text)}\t{pre_word_text}")
            





100%|██████████| 61/61 [00:00<00:00, 5530.99it/s]

2465	LOCATION-OTHER	68	79	PO BOX 1061
5243	LOCATION-OTHER	502	513	PO Box 1013
file37972	LOCATION-OTHER	493	590	&quot;UTERUS, PERITONEAL DISEASE, SPLEEN, OMENTUM, TUBES, OVARIES, SMALL BOWEL, GALLBLADDER&quot;
file37972	LOCATION-OTHER	5346	5373	&quot;RIGHT DIAPHRAGM&quot;
file41008	LOCATION-OTHER	476	520	&quot;SENTINEL NODE BIOPSY LEFT BREAST&quot;
file41008	LOCATION-OTHER	656	676	&quot;Negative&quot;
file41008	LOCATION-OTHER	722	765	&quot;WIDE LOCAL EXCISION LEFT BREAST&quot;
file41008	LOCATION-OTHER	2127	2171	&quot;LEFT AXILLARY TISSUE LEFT BREAST&quot;
file41429	LOCATION-OTHER	393	459	&quot;RE-EXCISION MARGINS INFERIOR AND SUPERIOR RIGHT BREAST&quot;
file41429	LOCATION-OTHER	460	522	&quot;Short - superior, Medium - medial, Long - lateral.&quot;
file42064	LOCATION-OTHER	385	482	&quot;RE-EXCISION OF ANTEROMEDIAL MARGIN, Short - superior, Medium - medial, Long - lateral&quot;
file50077	LOCATION-OTHER	308	389	&quot;OMENTUM, RIGHT COLON, SPLEEN, GALLBLADDER, RECTUM, PERITONEAL DISEASE&quo

In [238]:
with open('./Search_Phi_Result/FULL/Search_Phi_Full_Result.txt','w', encoding='utf-8') as f:
        for final_pre in final_pre_text:
            f.write(final_pre)
            f.write('\n')